In [1]:
import numpy as py
import pandas as pd
import gspread
import time

In [2]:
sa=gspread.service_account(filename='key.json')
sh=sa.open("Participant's Resources")
wks=sh.worksheet("Sheet1")
part=pd.DataFrame(wks.get_all_records())

In [3]:
# link for Participant resourses online 
# https://docs.google.com/spreadsheets/d/1sVnQPnImfx_qOFvGFkObnec1r7LfQ7WNAnuKU5JTQTA/edit?usp=sharing
# download 
# https://docs.google.com/spreadsheets/d/1sVnQPnImfx_qOFvGFkObnec1r7LfQ7WNAnuKU5JTQTA/gviz/tq?tqx=out:csv

# Link for Planet from planet distance online
# https://docs.google.com/spreadsheets/d/1iUknau0-Rv9UZ1xC_pa6ludZcrWx7BOaVCQHl7J0Vrk/edit?usp=sharing\
# Download
# https://docs.google.com/spreadsheets/d/1iUknau0-Rv9UZ1xC_pa6ludZcrWx7BOaVCQHl7J0Vrk/gviz/tq?tqx=out:csv

In [4]:
planet=pd.read_csv('https://docs.google.com/spreadsheets/d/1iUknau0-Rv9UZ1xC_pa6ludZcrWx7BOaVCQHl7J0Vrk/gviz/tq?tqx=out:csv')
part=pd.DataFrame(wks.get_all_records())


In [5]:
def details():
    planet=pd.read_csv('https://docs.google.com/spreadsheets/d/1iUknau0-Rv9UZ1xC_pa6ludZcrWx7BOaVCQHl7J0Vrk/gviz/tq?tqx=out:csv')
    part=pd.DataFrame(wks.get_all_records())
    return planet,part



In [6]:
#col=wks.find('test').col
#wks.update_cell(2,col,'hello')

In [7]:
# A - Participants
# B - Total Time
# C - Speed
# D - Oxygen
# E - LHydrogen
# F - Water
# G - Food
# H - T_Oxygen // Time for mining Oxygen
# I - T_Hydrogen // Time for mining Hydrogen
# J - Hunt animal
# K - Hunt Seafood
# L - Current Planet

In [8]:
def p_to_p(p,pl):
    col='B'+str(p+1)
    col1='D'+str(p+1)
    col2='G'+str(p+1)
    col3='F'+str(p+1)
    col4='H'+str(p+1)
    col5='E'+str(p+1)
    c_pl=wks.acell('L'+str(p+1)).value
    time=float(planet.loc[planet['Planets']==c_pl,pl].iloc[0])/4
    wks.update(col,float(float(wks.acell(col).value)-time))
    wks.update(col1,float(wks.acell(col1).value)-float(float(time)*0.09))
    wks.update(col2,float(wks.acell(col2).value)-float(2*(float(time))))
    wks.update(col3,float(wks.acell(col3).value)-float(time/50))
    wks.update(col5,float(wks.acell(col5).value)-float(time*0.12))
#p_to_p(1,'Kepler 283 c')


In [9]:
def ignition(p,pl):
    c_pl=wks.acell('L'+str(p+1)).value

    ig=0.25+0.25*float(planet.loc[planet['Planets']==wks.acell('L'+str(p+1)).value,'Gravity'].iloc[0])
    fuel=(0.06*float(planet.loc[planet['Planets']==c_pl,pl]))+ig
    col= 'E'+str(p+1)
    if(pd.isna(ig)==False and pd.isna(fuel)==False):
        wks.update(col,float(wks.acell(col).value)-fuel)
    #p_to_p(p,pl)

#ignition(2,'Earth')

In [10]:
def O2(p,pl,amnt):
    col1='D'+str(p+1)
    col2='B'+str(p+1)
    col3='F'+str(p+1)
    time=(amnt/400)*(planet.loc[planet['Planets']==pl,'Relative_time'].iloc[0])
    wks.update(col2,float(wks.acell(col2).value)-time)
    wks.update(col1,float(wks.acell(col1).value)+(0.7*(0.4*amnt)))
    wks.update(col3,float(wks.acell(col3).value)+(0.3*(0.4*amnt)))

#O2(2,'Proxima Centauri b',100)

In [11]:
def H(p,pl,amnt):
    col1='E'+str(p+1)
    col2='B'+str(p+1)
    time=(amnt/200)*(planet.loc[planet['Planets']==pl,'Relative_time'].iloc[0])
    wks.update(col2,float(wks.acell(col2).value)-time)
    wks.update(col1,float(wks.acell(col1).value)+(0.2*amnt))

#H(2,'Proxima Centauri b',10)


In [12]:
def hunt_animal(p,pl,amnt):
    col1='G'+str(p+1)
    col2='B'+str(p+1)
    time=(amnt/10)*(planet.loc[planet['Planets']==pl,'Relative_time'].iloc[0])
    wks.update(col2,float(wks.acell(col2).value)-time)
    wks.update(col1,float(wks.acell(col1).value)+(amnt))
#hunt_animal(1,'Proxima Centauri',10)
    

In [13]:
def hunt_seafood(p,pl,amnt):
    col1='G'+str(p+1)
    col2='B'+str(p+1)
    time=(amnt/30)*(planet.loc[planet['Planets']==pl,'Relative_time'].iloc[0])
    wks.update(col2,float(wks.acell(col2).value)-time)
    wks.update(col1,float(wks.acell(col1).value)+(amnt))

In [14]:
from pywebio import start_server
from pywebio.input import *
from pywebio.output import *
import tornado.ioloop
import threading


In [15]:
def updated_dataset(n):
    index=planet.index[planet['Planets']==wks.acell('L'+str(n+1)).value][0]
    print(index)
    limit=planet[0:index+7]
    return limit

In [16]:
def planet_list(n):
    pl=updated_dataset(n)
    res=[]
    name=wks.acell('L'+str(n+1)).value
    for i in pl.iloc[:,0]:
        if(i!=name):
            res.append(i)
    return res

In [17]:
def datasets(n):
    planet,part=details()
    pl=wks.acell('L'+str(n+1)).value
    put_html("<style> body{background-image:url(https://cdn.pixabay.com/photo/2016/10/20/18/35/earth-1756274_1280.jpg);background-repeat:repeat-y;}</style>")
    put_html(f"<h1><b style='color:white;'>Group {n}</b></h1><h4 style='text-align:right; color:white;'>Lives Left: {wks.acell('M'+str(n+1)).value}</h4><br>")
    put_html("<b style='color:white'>Participant's Resources</b>")
    put_html(pd.DataFrame(part.iloc[n-1]))
    put_html("<br><br><b style='color:white'>Current Planet Details</b>")
    put_html(pd.DataFrame(planet[planet['Planets']==pl]))
    planet=updated_dataset(n)
    put_html("<br><b style='color:white'>Planets you can travel to</b>")
    put_html(pd.DataFrame(planet))

In [18]:
def bars():
    put_processbar('bar')
    for i in range(1, 11):
       set_processbar('bar', i / 10)
       time.sleep(0.05)

In [19]:
import asyncio 

In [20]:
def reset():
    for p in range(1,15):
        col1='B'+str(p+1)
        c2='D'+str(p+1)
        c3='E'+str(p+1)
        c4='F'+str(p+1)
        c5='G'+str(p+1)
        c6='L'+str(p+1)
        c7='M'+str(p+1)
        wks.update(col1,float(80))
        wks.update(c2,float(29))
        wks.update(c3,float(100))
        wks.update(c4,float(10))
        wks.update(c5,float(50000))
        wks.update(c6,'Earth')
        wks.update(c7,int(3))
#reset()

In [21]:
def status(p):
    col1='B'+str(p+1)
    c2='D'+str(p+1)
    c3='E'+str(p+1)
    c4='F'+str(p+1)
    c5='G'+str(p+1)
    if((float(wks.acell(col1).value)<=0) or (float(wks.acell(c2).value)<=0) or (float(wks.acell(c3).value)<=0) or (float(wks.acell(c4).value)<=0) or (float(wks.acell(c5).value)<=0) ):
        wks.update(col1,float(2000))
        wks.update(c2,float(20))
        wks.update(c3,float(80))
        wks.update(c4,float(8))
        wks.update(c5,float(3000))
        return False
    return True
#status(1)
    

In [22]:
def main(group):
    n=group
    names=planet_list(n)
    pname=select('Select the planet you want to travel to: ',names)
    bars()
    ignition(n,wks.acell('L'+str(n+1)).value)
    p_to_p(n,pname)
    if(float(wks.acell('B'+str(n+1)).value)<=0):
        return 1
    else:
        wks.update('L'+str(n+1),pname)
    bars()
    clear()
    datasets(n)
    oxy=abs(input("Enter Amount of Oxygen to be mined: ",type=FLOAT))
    O2(n,pname,oxy)
    clear()
    bars()
    datasets(n)
    Hyd=abs(float(input("Enter Amount of Hydrogen to be mined: ")))
    H(n,pname,Hyd)
    clear()
    bars()
    datasets(n)
    if(planet.loc[planet['Planets']==wks.acell('L'+str(n+1)).value,'Land Animals'].iloc[0]=='Yes'):
        animal=abs(float(input("Enter how many Kgs of land animal you want to hunt: ")))
        hunt_animal(n,pname,animal)
        clear()
        bars()
        datasets(n)
    if(planet.loc[planet['Planets']==wks.acell('L'+str(n+1)).value,'Marine Life'].iloc[0]=='Yes'):
        sea=abs(float(input("Enter how many Kgs of Sea animal you want to hunt: ")))
        hunt_seafood(n,pname,sea)
        clear()
        bars()
        datasets(n)
    return select("0- If you decide to settle with planet  1- If you want to travel further: ",[0,1])
        

In [23]:
import time

In [24]:
def index():
    group=select('Select Group Number: ',[i for i in range(1,16)])
    psw=input('Enter Password: ',type=PASSWORD)
    start=0
    if(psw==wks.acell('N'+str(group+1)).value):
        start=1
        put_html("<style>body{background-image:url(https://c4.wallpaperflare.com/wallpaper/853/1009/106/simple-simple-background-minimalism-black-background-wallpaper-preview.jpg); background-size:cover; background-repeat:no-repeat;} h1,h2{justify-content:center; align-items:center; text-align:center; color:White;}</style><body><h1>Welcome To ORBIS ALTER</h1><h2>Presented by Lit-DacXSeds </h2></body>")
        time.sleep(4)
        clear()
    pl=pd.read_csv('https://docs.google.com/spreadsheets/d/1YNYAu_QMM0MQRWDKUQXU6w_PasVvzBeF7qkw0ea8Mzo/gviz/tq?tqx=out:csv')
    while(int(wks.acell('M'+str(group+1)).value) > 0 and start==1):
        flag=1
        print('Start1: ',start)
        while(flag!=0):
            datasets(group)
            flag=main(group)
            if(status(group)==False):
                flag=0
                break
        if(wks.acell('L'+str(group+1)).value==pl['Lme'].loc[0]):    
            clear()
            wks.update('L'+str(group+1),"WON")
            put_html("<style> body{background-image:url(https://cdn.pixabay.com/photo/2016/10/20/18/35/earth-1756274_1280.jpg);} h1{justify-content:center; align-items:center; text-align:center; color:white}</style><body><h1>YOU WON!!!</h1>")    
            break
        
        elif(flag==0 and start==1):
            clear()
            ccol='M'+str(group+1)
            wks.update(ccol,int(wks.acell(ccol).value)-1)
            put_html("<style> body{background-image:url(https://cdn.pixabay.com/photo/2016/10/20/18/35/earth-1756274_1280.jpg);} h1{justify-content:center; align-items:center; text-align:center; color:white}</style><body>"f"<h1>You Have {wks.acell(ccol).value} lives left </h1>")  
            time.sleep(3)
            clear()
            
        elif(wks.acell('M'+str(group+1)).value==0 and start==1):
            clear()
            wks.update('L'+str(group+1),"LOST")
            put_html("<style> body{background-image:url(https://cdn.pixabay.com/photo/2016/10/20/18/35/earth-1756274_1280.jpg);} h1{justify-content:center; align-items:center; text-align:center; color:white}</style><body><h1>Sorry,You Lost</h1>")  
            time.sleep(5)  

    if(start==0):
        put_html("<style> body{background-image:url(https://cdn.pixabay.com/photo/2016/10/20/18/35/earth-1756274_1280.jpg);} h1,h2{justify-content:center; align-items:center; text-align:center; color:white}</style><body><h1>Wrong Password</h1><h2>Refresh to Re-Enter Credentials</h2>")    
        
   # loop.run_forever()
    

    
loop = asyncio.new_event_loop()    
asyncio.set_event_loop(loop)
# Start the PyWebIO server
#start_server(index)

# Stop the event loop
#tornado.ioloop.IOLoop.current().stop()

#import asyncio
#import threading

#def run_event_loop():
#    loop = asyncio.new_event_loop()
#    asyncio.set_event_loop(loop)
#    loop.run_forever()

#def run_event_loop():
#    start_server(index)

#thread = threading.Thread(target=run_event_loop)
#thread.start()


In [25]:
start_server(index)

Running on all addresses.
Use http://192.168.0.104:64081/ to access the application


RuntimeError: This event loop is already running

Start1:  1
7
7


In [ ]:
for i 